# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [18]:
# imports
import os
import json
import requests
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [55]:
# constants

MODEL = 'gpt-4o-mini'
# MODEL = 'llama3.2'

In [24]:
# set up environment
load_dotenv(override=True)
api_key=os.getenv("OPENAI_API_KEY")
if api_key:
    print("Keys are not available")
    print(api_key[:6] + "*" * (len(api_key) - 6))
else:
    print("API keys are not setup in .env")


Keys are not available
sk-pro**************************************************************************************************************************************************************


In [15]:
# here is the question; type over this to ask something new

question = """
This is to review the technical document of any integration and come up with the help guide. This will help organization to know if the
vendor is suitable for their use case or not. 

For example if a vendor is to be used for payout. We need have following:-
1. Authorization
2. Payee/Beneficiary addition
3. Bank Add/Bank withdrawl OR crypto withdrawl
4. Status check
5. Webhook
6. Fee configuration
"""

In [38]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class WebsiteReview:
    """
    A utility class to extract website technical documents
    """
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        #print(response.content)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        # print(soup.prettify())
        # print(soup.title.string)
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

    

In [64]:
system_prompt = "You are provided with a list of links found on a techncial integratation webpage. \
You are able to decide which of the links would be most relevant to go through for integrating vendor as a payout provider, \
such as links to an Authorization page, or a Beneficiary KYC and KYB, or Bank addition page or crypto/bank transfer page.\n"
system_prompt += "You should respond in JSON as in this example:"
system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [71]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for integration as payout vendor, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [72]:
# print(get_links_user_prompt(techWebsite))
def ai_link_filter(url):
    website = WebsiteReview(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)
    
def get_links_content(url):
    result = "Landing page:\n"
    result += WebsiteReview(url).get_contents()
    links = ai_link_filter(url)
    # print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += WebsiteReview(link["url"]).get_contents()
    return result

def get_technical_review_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its technical integration and other relevant pages; use this information to build a short step by step technical integration summary in markdown.\n"
    user_prompt += get_links_content(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [73]:
def create_tech_summary(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_technical_review_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    
create_tech_summary("Rocketfuel", "https://docs.rocketfuelblockchain.com/")

Found links: {'links': [{'type': 'payout overview', 'url': 'https://docs.rocketfuelblockchain.com/developer-guides/api-reference/payout/overview'}, {'type': 'add payee', 'url': 'https://docs.rocketfuelblockchain.com/developer-guides/api-reference/payout/add-payee'}, {'type': 'payout api guide', 'url': 'https://docs.rocketfuelblockchain.com/developer-guides/api-reference/payout/api-guide'}, {'type': 'payout webhooks', 'url': 'https://docs.rocketfuelblockchain.com/developer-guides/api-reference/payout/webhooks'}, {'type': 'KYC verification', 'url': 'https://docs.rocketfuelblockchain.com/integrations/rocketfuel-integration/how-to-guide/kyc-verification'}, {'type': 'bank ACH payments user guide', 'url': 'https://docs.rocketfuelblockchain.com/user-guide-and-help-videos/merchant-user-guide/bank-ach-payments'}, {'type': 'bank ACH payments shopper user guide', 'url': 'https://docs.rocketfuelblockchain.com/user-guide-and-help-videos/shopper-user-guide/bank-ach-payments'}, {'type': 'merchant onb

```json
{
    "links": [
        {"type": "Payout Overview", "url": "https://rocketfuel.com/payout/overview"},
        {"type": "Add Payee", "url": "https://rocketfuel.com/payout/add-payee"},
        {"type": "KYC Verification", "url": "https://rocketfuel.com/integration/kyc-verification"},
        {"type": "API Guide", "url": "https://rocketfuel.com/payout/api-guide"},
        {"type": "Webhooks", "url": "https://rocketfuel.com/payout/webhooks"},
        {"type": "RocketFuel Integration", "url": "https://rocketfuel.com/integration/overview"},
        {"type": "How to Use Testnet for Transactions", "url": "https://rocketfuel.com/integration/testnet"}
    ]
}
```

In [ ]:
# Get Llama 3.2 to answer